# Lab 2 - A security use case

In [1]:
from agents import Agent, Runner, function_tool
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import Literal

load_dotenv(override=True)

MODEL = "gpt-4.1-mini"

## First, go check out the module `airline.py`

Then we'll try it out:

In [2]:
!uv run airline.py

^C


In [3]:
@function_tool
def read_file(filename: str) -> str:
    """ Read the given file and return its contents.

    Args:
        filename: The name of the file to read
    """
    with open(filename, "r") as f:
        result = f.read()
    return result

In [4]:
read_file

FunctionTool(name='read_file', description='Read the given file and return its contents.', params_json_schema={'properties': {'filename': {'description': 'The name of the file to read', 'title': 'Filename', 'type': 'string'}}, 'required': ['filename'], 'title': 'read_file_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000021353F42340>, strict_json_schema=True, is_enabled=True)

In [5]:
class Vulnerability(BaseModel):
    title: str = Field(description="A title for the vulnerability")
    description: str = Field(description="A description of the vulnerability")
    code: str = Field(description="A snippet of code that is vulnerable")
    suggested_fix: str = Field(description="How the code could be fixed")
    cvss_score: float = Field(description="The CVSS score of the vulnerability")
    severity: Literal["critical", "high", "medium", "low"] = Field(description="The severity of the vulnerability")

    def __str__(self):
        return f"""{self.title} - ({self.severity.upper()})
Description: {self.description}
Code: {self.code}
Suggested Fix: {self.suggested_fix}
CVSS Score: {self.cvss_score}
"""


class SecurityReport(BaseModel):
    vulnerabilities: list[Vulnerability] = Field(description="A list of vulnerabilities found in the file")

    def __str__(self):
        if self.vulnerabilities:
            return f"{len(self.vulnerabilities)} vulnerabilities found:\n" + "\n".join(str(v) for v in self.vulnerabilities)
        else:
            return "No vulnerabilities found"


In [6]:
instructions = """
You are a cybersecurity researcher. You are given the name of a python file to analyze.
You read the file, and analyze it for security vulnerabilities.
You reply with all your findings; include medium and low severity vulnerabilities in addition to any high or critical ones.
"""

agent = Agent(name="Security Researcher", instructions=instructions, model="gpt-4.1", tools=[read_file], output_type=SecurityReport)
result = await Runner.run(agent, input="Please analyze the file airline.py")
report = result.final_output_as(SecurityReport)



In [7]:
print(report)

5 vulnerabilities found:
SQL Injection in get_ticket_price - (HIGH)
Description: The get_ticket_price function constructs SQL queries by directly interpolating user input (the city name) into the SQL query string. This allows an attacker to craft input that could modify the behavior of the query, resulting in unauthorized disclosure, modification, or deletion of data in the database.
Code: query = f"SELECT price FROM prices WHERE city = '{city.lower()}'"
Suggested Fix: Use parameterized queries to safely handle user input: 
cursor.execute("SELECT price FROM prices WHERE city = ?", (city.lower(),))
CVSS Score: 7.5

Arbitrary Code Execution via eval in calculate - (CRITICAL)
Description: The calculate function uses Python's eval on user-provided input (the expr string) without any validation or restriction. This allows an attacker to execute arbitrary code on the server, potentially compromising the system.
Code: return str(eval(expr))
Suggested Fix: Avoid using eval on user input. For a

## That's it! We just built a simple business use case with an Agent, a Tool and Structured Outputs